# GPU

In [ ]:
import tensorflow as tf

print(tf.__version__)

tf.config.experimental.list_physical_devices(device_type=None)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Original BILSTM

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# =======================
# 1️⃣ GPU Memory Optimization
# =======================
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU Memory Growth Enabled")
    except RuntimeError as e:
        print(e)

# Enable mixed precision
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# =======================
# 2️⃣ Load Features with Sliding Windows
# =======================
base_path = "/mnt/d/Mass Projects/Video_Summ/new_dataset_features_npy1"
train_path = os.path.join(base_path, "train")
valid_path = os.path.join(base_path, "valid")

SEQ_LEN = 150     # ✅ smaller window to avoid OOM
STRIDE = 125      # ✅ overlap to preserve temporal context
FEATURE_DIM = 2048

def load_features(directory):
    all_windows = []
    video_files = sorted(os.listdir(directory))
    for file in video_files:
        file_path = os.path.join(directory, file)
        features = np.load(file_path)  # shape: (1750, 2048)

        # Create sliding windows
        for start in range(0, len(features) - SEQ_LEN + 1, STRIDE):
            window = features[start:start+SEQ_LEN]
            all_windows.append(window)
    return np.array(all_windows, dtype=np.float32)  # shape: (num_windows, SEQ_LEN, FEATURE_DIM)

print("📥 Loading dataset...")
train_features = load_features(train_path)
valid_features = load_features(valid_path)
print("Train windows:", train_features.shape)
print("Valid windows:", valid_features.shape)

NUM_EPOCHS = 100
BATCH_SIZE = 1
LEARNING_RATE = 1e-7
MODEL_SAVE_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Models_2/SUMME/video_summarization_4-heads_15k.keras"
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)

# =======================
# 3️⃣ Define BiLSTM + Multi-Head Attention Model
# =======================
class BiLSTMAttentionModel(models.Model):
    def __init__(self, feature_dim, num_heads=4, lstm_units=64, **kwargs):
        super(BiLSTMAttentionModel, self).__init__(**kwargs)
        self.feature_dim = feature_dim
        self.num_heads = num_heads
        self.lstm_units = lstm_units

        self.bilstm = layers.Bidirectional(
            layers.LSTM(lstm_units, return_sequences=True)
        )
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=feature_dim // num_heads
        )
        self.reconstruction_layer = layers.TimeDistributed(
            layers.Dense(feature_dim, activation="linear")
        )

    def call(self, inputs):
        x = self.bilstm(inputs)
        x = self.attention(x, x)
        reconstructed = self.reconstruction_layer(x)
        return reconstructed

    def get_config(self):
        config = super(BiLSTMAttentionModel, self).get_config()
        config.update({
            "feature_dim": self.feature_dim,
            "num_heads": self.num_heads,
            "lstm_units": self.lstm_units
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

# =======================
# 4️⃣ Build + Compile Model
# =======================
model = BiLSTMAttentionModel(feature_dim=FEATURE_DIM)
model.build(input_shape=(None, SEQ_LEN, FEATURE_DIM))

model.compile(
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
    loss="mse",
    metrics=["mae"]
)

# =======================
# 5️⃣ Training
# =======================
print("🚀 Starting Training...")
history = model.fit(
    train_features, train_features,
    validation_data=(valid_features, valid_features),
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE
)

model.save(MODEL_SAVE_PATH)
print(f"✅ Model saved at {MODEL_SAVE_PATH}")

# =======================
# 6️⃣ Evaluation
# =======================
with tf.keras.utils.custom_object_scope({"BiLSTMAttentionModel": BiLSTMAttentionModel}):
    model = tf.keras.models.load_model(MODEL_SAVE_PATH)

print("🔄 Model loaded for evaluation.")

test_video = valid_features[0:1]  # (1, SEQ_LEN, 2048)
reconstructed_video = model.predict(test_video)
print("Reconstructed Features Shape:", reconstructed_video.shape)


# Evaluation

In [ ]:
import os
import numpy as np
import tensorflow as tf
import scipy.io
import ruptures as rpt
import pandas as pd
import gc

# ========= 1. Load Model =========
class BiLSTMAttentionModel(tf.keras.Model):
    def __init__(self, feature_dim=2048, num_heads=2, lstm_units=128, **kwargs):
        super(BiLSTMAttentionModel, self).__init__(**kwargs)
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(lstm_units, return_sequences=True))
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=feature_dim // num_heads)
        self.reconstruction_layer = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(feature_dim, activation="linear"))

    def call(self, inputs):
        x = self.bilstm(inputs)
        x = self.attention(x, x)
        return self.reconstruction_layer(x)

# MODEL_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/bilstm_multi_2-heads_60k.keras"
MODEL_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Final/Models/bilstm_multi_4-heads_60k_2.keras"
with tf.keras.utils.custom_object_scope({"BiLSTMAttentionModel": BiLSTMAttentionModel}):
    model = tf.keras.models.load_model(MODEL_PATH)
print("\n✅ Model loaded successfully")

# ========= 2. Helper Functions =========
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def format_segments(segments):
    return [f"({s})" if s == e else f"({s}–{e})" for s, e in segments]

def print_segments_columnwise(segments, items_per_line=10):
    for i in range(0, len(segments), items_per_line):
        print("  " + "  ".join(segments[i:i + items_per_line]))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    matched = []
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                matched.append(m_seg)
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall, matched

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    change_points = algo.predict(pen=penalty)
    change_points = [int(cp) for cp in change_points]
    scene_segments = []
    start_frame = 0
    for cp in change_points:
        scene_segments.append((start_frame, cp - 1))
        start_frame = cp
    if scene_segments[-1][1] < features.shape[0] - 1:
        scene_segments.append((start_frame, features.shape[0] - 1))
    return scene_segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]

# ========= 3. Process Videos =========
video_list = [
    "Jumps", "Base Jumping", "Air_Force_One", "Bearpark_climbing", "Bike Polo",
    "Bus_in_Rock_Tunnel", "car_over_camera", "Car_railcrossing", "Cockpit_Landing", "Cooking",
    "Eiffel Tower", "Excavators river crossing", "Fire Domino", "Kids_playing_in_leaves",
    "Notre_Dame", "Paintball", "paluma_jump", "playing_ball", "Playing_on_water_slide",
    "Saving dolphines", "Scuba", "St Maarten Landing", "Statue of Liberty", "Valparaiso_Downhill"
]

FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/extracted_features/"
MAT_DIR = "/mnt/d/Mass Projects/Video_Summ/mat/"
CHUNK_SIZE = 500
PENALTY = 500
FPS = 25
SUMMARY_EXCEL = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Final/Results/SUMME/bilstm_multi_4-heads_60k_2.xlsx"

with pd.ExcelWriter(SUMMARY_EXCEL, engine="xlsxwriter") as writer:
    summary_data = []

    for VIDEO_NAME in video_list:
        print(f"\n==============================\n🚀 Processing video: {VIDEO_NAME}\n==============================")

        feats_path = os.path.join(FEATURES_DIR, f"{VIDEO_NAME}.npy")
        mat_path = os.path.join(MAT_DIR, f"{VIDEO_NAME}.mat")
        if not os.path.exists(feats_path) or not os.path.exists(mat_path):
            print("❌ Skipping, missing file.")
            continue

        raw_feats = np.load(feats_path)
        num_frames = raw_feats.shape[0]
        video_feats = raw_feats[np.newaxis, ...]

        recon = np.zeros_like(video_feats)
        for i in range((num_frames + CHUNK_SIZE - 1) // CHUNK_SIZE):
            s, e = i * CHUNK_SIZE, min((i + 1) * CHUNK_SIZE, num_frames)
            recon[:, s:e, :] = model.predict(video_feats[:, s:e, :], verbose=0)
        print("✅ Features reconstructed")

        errs = np.mean(np.abs(video_feats - recon), axis=2).flatten()
        imp_scores = (errs - errs.min()) / (errs.max() - errs.min())

        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        mat = scipy.io.loadmat(mat_path)
        user_scores = mat['user_score']
        num_users = user_scores.shape[1]
        all_user_metrics = []

        for u in range(num_users):
            usr = user_scores[:, u]
            nonzero_indices = np.where(usr > 0)[0]
            user_segments = get_segments_from_indices(nonzero_indices.tolist())

            f1, precision, recall, _ = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=VIDEO_NAME[:31], index=False)

        summary_data.append({
            "Video Name": VIDEO_NAME,
            "KTS Segments": len(model_segments),
            "Knapsack Selected": len(selected_model_segments),
            "Penalty": PENALTY,
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max(),
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
        })

        del video_feats, recon
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ All results saved to: {SUMMARY_EXCEL}")


# TVSUM EVALUATION

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
import gc
import ruptures as rpt

# ========= 1. Load Model =========
class BiLSTMAttentionModel(tf.keras.Model):
    def __init__(self, feature_dim=2048, num_heads=8, lstm_units=128, **kwargs):
        super(BiLSTMAttentionModel, self).__init__(**kwargs)
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(lstm_units, return_sequences=True))
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=feature_dim // num_heads)
        self.reconstruction_layer = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(feature_dim, activation="linear"))

    def call(self, inputs):
        x = self.bilstm(inputs)
        x = self.attention(x, x)
        return self.reconstruction_layer(x)

MODEL_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/bilstm_multi_8-heads_60k.keras"
with tf.keras.utils.custom_object_scope({"BiLSTMAttentionModel": BiLSTMAttentionModel}):
    model = tf.keras.models.load_model(MODEL_PATH)
print("\n✅ Model loaded successfully")

# ========= 2. Helper Functions =========
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    change_points = algo.predict(pen=penalty)
    change_points = [int(cp) for cp in change_points]
    scene_segments = []
    start_frame = 0
    for cp in change_points:
        scene_segments.append((start_frame, cp - 1))
        start_frame = cp
    if scene_segments[-1][1] < features.shape[0] - 1:
        scene_segments.append((start_frame, features.shape[0] - 1))
    return scene_segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]

# ========= 3. TVSum Paths =========
FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/TVSUM/RandomSplit_Dataset/resnet_npy/extracted_features_25fps_videos"
MAT_FILE_PATH = "/mnt/d/Mass Projects/Video_Summ/ydata-tvsum50-v1_1/ydata-tvsum50-matlab/matlab/ydata-tvsum50.mat"
EXCEL_OUT = "/mnt/d/Mass Projects/Video_Summ/RESULTS/RESULTS/TVSUM/bilstm_multi_8-heads_60k.xlsx"
FPS = 25
PENALTY = 500
CHUNK_SIZE = 500

# ========= 4. Load .mat TVSum =========
with h5py.File(MAT_FILE_PATH, "r") as mat_data:
    tvsum50_group = mat_data["tvsum50"]

    def decode_str(ref):
        return "".join(map(chr, mat_data[ref][()].flatten())).replace("\x00", "")

    video_names = [decode_str(ref) for ref in tvsum50_group["video"][:, 0]]
    user_annos = [mat_data[ref][:] for ref in tvsum50_group["user_anno"][:, 0]]
    nframes_list = [int(mat_data[ref][()][0, 0]) for ref in tvsum50_group["nframes"][:, 0]]

# ========= 5. Process Videos =========
with pd.ExcelWriter(EXCEL_OUT, engine="xlsxwriter") as writer:
    summary_data = []

    for idx, video_name in enumerate(video_names):
        print(f"\n🚀 Processing video: {video_name}")

        feature_path = os.path.join(FEATURES_DIR, f"{video_name}.npy")
        if not os.path.exists(feature_path):
            print(f"❌ Missing feature file: {video_name}")
            continue

        raw_feats = np.load(feature_path)
        num_frames = raw_feats.shape[0]
        video_feats = raw_feats[np.newaxis, ...]

        recon = np.zeros_like(video_feats)
        for i in range((num_frames + CHUNK_SIZE - 1) // CHUNK_SIZE):
            s, e = i * CHUNK_SIZE, min((i + 1) * CHUNK_SIZE, num_frames)
            recon[:, s:e, :] = model.predict(video_feats[:, s:e, :], verbose=0)

        errs = np.mean(np.abs(video_feats - recon), axis=2).flatten()
        imp_scores = (errs - errs.min()) / (errs.max() - errs.min())

        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        user_scores = user_annos[idx]
        num_users = user_scores.shape[0]
        all_user_metrics = []

        for u in range(num_users):
            user_score = user_scores[u]
            k = int(np.floor(num_frames * 0.15))
            selected_indices = np.argsort(user_score)[-k:]
            user_segments = get_segments_from_indices(selected_indices.tolist())

            f1, precision, recall = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=video_name[:31], index=False)

        summary_data.append({
            "Video": video_name,
            "KTS Segments": len(model_segments),
            "Selected": len(selected_model_segments),
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max()
        })

        del video_feats, recon
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ TVSum results saved to: {EXCEL_OUT}")


# NO ATTENTION TRAINING

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# =======================
# 1️⃣ GPU Memory Optimization
# =======================
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # ✅ Prevents full GPU memory allocation
        print("✅ GPU Memory Growth Enabled")
    except RuntimeError as e:
        print(e)

# Enable mixed precision training to reduce memory usage
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# =======================
# 2️⃣ Load Pre-Extracted Features
# =======================
base_path = "/mnt/d/Mass Projects/Video_Summ/new_dataset_features_npy_3000_1"
train_path = os.path.join(base_path, "train")
valid_path = os.path.join(base_path, "valid")

def load_features(directory):
    feature_list = []
    video_files = sorted(os.listdir(directory))  # Ensure correct order
    for file in video_files:
        file_path = os.path.join(directory, file)
        features = np.load(file_path)  # Shape: (750, 2048)
        feature_list.append(features)
    return np.array(feature_list)  # Shape: (num_videos, seq_len, feature_dim)

train_features = load_features(train_path)  # (num_train_videos, 750, 2048)
valid_features = load_features(valid_path)  # (num_valid_videos, 750, 2048)

SEQ_LEN = train_features.shape[1]  # 750
FEATURE_DIM = train_features.shape[2]  # 2048
NUM_EPOCHS = 50
BATCH_SIZE = 1  # Reduce if needed to avoid OOM
LEARNING_RATE = 1e-5
MODEL_SAVE_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/bilstm_NO_attention_60k.keras"

os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)

# =======================
# 3️⃣ Define BiLSTM-Only Model (No Attention)
# =======================
class BiLSTMModel(models.Model):
    def __init__(self, feature_dim, lstm_units=128, **kwargs):
        super(BiLSTMModel, self).__init__(**kwargs)
        self.feature_dim = feature_dim
        self.lstm_units = lstm_units

        # BiLSTM Layer
        self.bilstm = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True))

        # Reconstruction Layer (to reconstruct original input)
        self.reconstruction_layer = layers.TimeDistributed(layers.Dense(feature_dim, activation="linear"))

    def call(self, inputs):
        x = self.bilstm(inputs)  # BiLSTM processing
        reconstructed = self.reconstruction_layer(x)  # Reconstruct input
        return reconstructed

    def get_config(self):
        config = super(BiLSTMModel, self).get_config()
        config.update({
            "feature_dim": self.feature_dim,
            "lstm_units": self.lstm_units
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

# Initialize Model
model = BiLSTMModel(feature_dim=FEATURE_DIM)

# Explicitly Build Model Before Saving
model.build(input_shape=(None, SEQ_LEN, FEATURE_DIM))

# Compile Model
model.compile(optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
              loss="mse",  # Mean Squared Error for reconstruction loss
              metrics=["mae"])

# =======================
# 4️⃣ Train Model (Unsupervised)
# =======================
print("🚀 Starting Training...")
history = model.fit(train_features, train_features,  # Unsupervised: reconstruct input
                    validation_data=(valid_features, valid_features),
                    epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

# Save Model
model.save(MODEL_SAVE_PATH)
print(f"✅ Model saved at {MODEL_SAVE_PATH}")

# =======================
# 5️⃣ Evaluation (Inference)
# =======================
with tf.keras.utils.custom_object_scope({"BiLSTMModel": BiLSTMModel}):
    model = tf.keras.models.load_model(MODEL_SAVE_PATH)

print("🔄 Model loaded for evaluation.")

# Example evaluation on a validation video
test_video = valid_features[0:1]  # Shape: (1, 750, 2048)
reconstructed_video = model.predict(test_video)
print("Reconstructed Features Shape:", reconstructed_video.shape)


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# =======================
# 1️⃣ GPU Memory Optimization
# =======================
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU Memory Growth Enabled")
    except RuntimeError as e:
        print(e)

# Enable mixed precision
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# =======================
# 2️⃣ Load Features with Sliding Windows
# =======================
base_path = "/mnt/d/Mass Projects/Video_Summ/new_dataset_features_npy_1750"
train_path = os.path.join(base_path, "train")
valid_path = os.path.join(base_path, "valid")

SEQ_LEN = 150
STRIDE = 125
FEATURE_DIM = 2048

def load_features(directory):
    all_windows = []
    video_files = sorted(os.listdir(directory))
    for file in video_files:
        file_path = os.path.join(directory, file)
        features = np.load(file_path)  # shape: (1750, 2048)
        for start in range(0, len(features) - SEQ_LEN + 1, STRIDE):
            window = features[start:start+SEQ_LEN]
            all_windows.append(window)
    return np.array(all_windows, dtype=np.float32)

print("📥 Loading dataset...")
train_features = load_features(train_path)
valid_features = load_features(valid_path)
print("Train windows:", train_features.shape)
print("Valid windows:", valid_features.shape)

NUM_EPOCHS = 50
BATCH_SIZE = 1
LEARNING_RATE = 1e-5
MODEL_SAVE_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Models_2/SUMME/bilstm_no_attention_35k.keras"
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)

# =======================
# 3️⃣ Define BiLSTM Autoencoder (No Attention)
# =======================
class BiLSTMNoAttentionModel(models.Model):
    def __init__(self, feature_dim, lstm_units=64, **kwargs):
        super(BiLSTMNoAttentionModel, self).__init__(**kwargs)
        self.feature_dim = feature_dim
        self.lstm_units = lstm_units

        self.bilstm = layers.Bidirectional(
            layers.LSTM(lstm_units, return_sequences=True)
        )
        self.reconstruction_layer = layers.TimeDistributed(
            layers.Dense(feature_dim, activation="linear")
        )

    def call(self, inputs):
        x = self.bilstm(inputs)
        reconstructed = self.reconstruction_layer(x)
        return reconstructed

    def get_config(self):
        config = super(BiLSTMNoAttentionModel, self).get_config()
        config.update({
            "feature_dim": self.feature_dim,
            "lstm_units": self.lstm_units
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

# =======================
# 4️⃣ Build + Compile Model
# =======================
model = BiLSTMNoAttentionModel(feature_dim=FEATURE_DIM)
model.build(input_shape=(None, SEQ_LEN, FEATURE_DIM))

model.compile(
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
    loss="mse",
    metrics=["mae"]
)

# =======================
# 5️⃣ Training
# =======================
print("🚀 Starting Training (No Attention)...")
history = model.fit(
    train_features, train_features,
    validation_data=(valid_features, valid_features),
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE
)

model.save(MODEL_SAVE_PATH)
print(f"✅ Model saved at {MODEL_SAVE_PATH}")

# =======================
# 6️⃣ Evaluation
# =======================
with tf.keras.utils.custom_object_scope({"BiLSTMNoAttentionModel": BiLSTMNoAttentionModel}):
    model = tf.keras.models.load_model(MODEL_SAVE_PATH)

print("🔄 Model loaded for evaluation.")

test_video = valid_features[0:1]  # (1, SEQ_LEN, 2048)
reconstructed_video = model.predict(test_video)
print("Reconstructed Features Shape:", reconstructed_video.shape)


# NO Attention - SUMME

In [ ]:
import os
import numpy as np
import tensorflow as tf
import scipy.io
import ruptures as rpt
import pandas as pd
import gc

# ========= 1️⃣ Load No-Attention BiLSTM Model =========
class BiLSTMNoAttentionModel(tf.keras.Model):
    def __init__(self, feature_dim=2048, lstm_units=64, **kwargs):
        super(BiLSTMNoAttentionModel, self).__init__(**kwargs)
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(lstm_units, return_sequences=True)
        )
        self.reconstruction_layer = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(feature_dim, activation="linear")
        )

    def call(self, inputs):
        x = self.bilstm(inputs)
        return self.reconstruction_layer(x)

MODEL_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Models_2/SUMME/bilstm_no_attention_35k.keras"

with tf.keras.utils.custom_object_scope({"BiLSTMNoAttentionModel": BiLSTMNoAttentionModel}):
    model = tf.keras.models.load_model(MODEL_PATH)

print("\n✅ BiLSTM No-Attention model loaded successfully")

# ========= 2️⃣ Helper Functions =========
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    change_points = algo.predict(pen=penalty)
    change_points = [int(cp) for cp in change_points]
    segments = []
    start = 0
    for cp in change_points:
        segments.append((start, cp - 1))
        start = cp
    if segments[-1][1] < features.shape[0] - 1:
        segments.append((start, features.shape[0] - 1))
    return segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]

# ========= 3️⃣ Process Videos =========
video_list = [
    "Jumps", "Base Jumping", "Air_Force_One", "Bearpark_climbing", "Bike Polo",
    "Bus_in_Rock_Tunnel", "car_over_camera", "Car_railcrossing", "Cockpit_Landing", "Cooking",
    "Eiffel Tower", "Excavators river crossing", "Fire Domino", "Kids_playing_in_leaves",
    "Notre_Dame", "Paintball", "paluma_jump", "playing_ball", "Playing_on_water_slide",
    "Saving dolphines", "Scuba", "St Maarten Landing", "Statue of Liberty", "Valparaiso_Downhill"
]

FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/extracted_features/"
MAT_DIR = "/mnt/d/Mass Projects/Video_Summ/mat/"
CHUNK_SIZE = 500
PENALTY = 500
FPS = 25
SUMMARY_EXCEL = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Results_2/SUMME/bilstm_no_attention_35k.xlsx"

with pd.ExcelWriter(SUMMARY_EXCEL, engine="xlsxwriter") as writer:
    summary_data = []

    for VIDEO_NAME in video_list:
        print(f"\n==============================\n🚀 Processing video: {VIDEO_NAME}\n==============================")

        feats_path = os.path.join(FEATURES_DIR, f"{VIDEO_NAME}.npy")
        mat_path = os.path.join(MAT_DIR, f"{VIDEO_NAME}.mat")
        if not os.path.exists(feats_path) or not os.path.exists(mat_path):
            print("❌ Skipping, missing file.")
            continue

        raw_feats = np.load(feats_path)
        num_frames = raw_feats.shape[0]
        video_feats = raw_feats[np.newaxis, ...]

        # Reconstruction in chunks
        recon = np.zeros_like(video_feats)
        for i in range((num_frames + CHUNK_SIZE - 1) // CHUNK_SIZE):
            s, e = i * CHUNK_SIZE, min((i + 1) * CHUNK_SIZE, num_frames)
            recon[:, s:e, :] = model.predict(video_feats[:, s:e, :], verbose=0)
        print("✅ Features reconstructed")

        # Compute importance scores (based on reconstruction error)
        errs = np.mean(np.abs(video_feats - recon), axis=2).flatten()
        imp_scores = (errs - errs.min()) / (errs.max() - errs.min())

        # Segment detection using KTS
        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        # Knapsack summary selection (15% of total duration)
        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        # Load ground-truth user summaries
        mat = scipy.io.loadmat(mat_path)
        user_scores = mat['user_score']
        num_users = user_scores.shape[1]
        all_user_metrics = []

        for u in range(num_users):
            usr = user_scores[:, u]
            nonzero_indices = np.where(usr > 0)[0]
            user_segments = get_segments_from_indices(nonzero_indices.tolist())

            f1, precision, recall = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=VIDEO_NAME[:31], index=False)

        summary_data.append({
            "Video Name": VIDEO_NAME,
            "KTS Segments": len(model_segments),
            "Knapsack Selected": len(selected_model_segments),
            "Penalty": PENALTY,
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max(),
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
        })

        del video_feats, recon
        gc.collect()
        tf.keras.backend.clear_session()

    # Save summary sheet
    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ All results saved to: {SUMMARY_EXCEL}")


# NO Attention TVSUM

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
import gc
import ruptures as rpt

# ========= 1. Load Model (BiLSTM without Attention) =========
class BiLSTMNoAttentionModel(tf.keras.Model):
    def __init__(self, feature_dim=2048, lstm_units=128, **kwargs):
        super(BiLSTMNoAttentionModel, self).__init__(**kwargs)
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(lstm_units, return_sequences=True)
        )
        self.reconstruction_layer = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(feature_dim, activation="linear")
        )

    def call(self, inputs):
        x = self.bilstm(inputs)
        return self.reconstruction_layer(x)

MODEL_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/bilstm_no_attention_15k.keras"
with tf.keras.utils.custom_object_scope({"BiLSTMNoAttentionModel": BiLSTMNoAttentionModel}):
    model = tf.keras.models.load_model(MODEL_PATH)

print("\n✅ No-attention BiLSTM model loaded successfully")

# ========= 2. Helper Functions =========
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    change_points = algo.predict(pen=penalty)
    change_points = [int(cp) for cp in change_points]
    scene_segments = []
    start_frame = 0
    for cp in change_points:
        scene_segments.append((start_frame, cp - 1))
        start_frame = cp
    if scene_segments[-1][1] < features.shape[0] - 1:
        scene_segments.append((start_frame, features.shape[0] - 1))
    return scene_segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]

# ========= 3. TVSum Paths =========
FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/TVSUM/RandomSplit_Dataset/resnet_npy/extracted_features_25fps_videos"
MAT_FILE_PATH = "/mnt/d/Mass Projects/Video_Summ/ydata-tvsum50-v1_1/ydata-tvsum50-matlab/matlab/ydata-tvsum50.mat"
EXCEL_OUT = "/mnt/d/Mass Projects/Video_Summ/RESULTS/RESULTS/TVSUM/bilstm_no_attention_15k.xlsx"
FPS = 25
PENALTY = 500
CHUNK_SIZE = 500

# ========= 4. Load .mat TVSum =========
with h5py.File(MAT_FILE_PATH, "r") as mat_data:
    tvsum50_group = mat_data["tvsum50"]

    def decode_str(ref):
        return "".join(map(chr, mat_data[ref][()].flatten())).replace("\x00", "")

    video_names = [decode_str(ref) for ref in tvsum50_group["video"][:, 0]]
    user_annos = [mat_data[ref][:] for ref in tvsum50_group["user_anno"][:, 0]]
    nframes_list = [int(mat_data[ref][()][0, 0]) for ref in tvsum50_group["nframes"][:, 0]]

# ========= 5. Process Videos =========
with pd.ExcelWriter(EXCEL_OUT, engine="xlsxwriter") as writer:
    summary_data = []

    for idx, video_name in enumerate(video_names):
        print(f"\n🚀 Processing video: {video_name}")

        feature_path = os.path.join(FEATURES_DIR, f"{video_name}.npy")
        if not os.path.exists(feature_path):
            print(f"❌ Missing feature file: {video_name}")
            continue

        raw_feats = np.load(feature_path)
        num_frames = raw_feats.shape[0]
        video_feats = raw_feats[np.newaxis, ...]

        recon = np.zeros_like(video_feats)
        for i in range((num_frames + CHUNK_SIZE - 1) // CHUNK_SIZE):
            s, e = i * CHUNK_SIZE, min((i + 1) * CHUNK_SIZE, num_frames)
            recon[:, s:e, :] = model.predict(video_feats[:, s:e, :], verbose=0)

        errs = np.mean(np.abs(video_feats - recon), axis=2).flatten()
        imp_scores = (errs - errs.min()) / (errs.max() - errs.min())

        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        user_scores = user_annos[idx]
        num_users = user_scores.shape[0]
        all_user_metrics = []

        for u in range(num_users):
            user_score = user_scores[u]
            k = int(np.floor(num_frames * 0.15))
            selected_indices = np.argsort(user_score)[-k:]
            user_segments = get_segments_from_indices(selected_indices.tolist())

            f1, precision, recall = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=video_name[:31], index=False)

        summary_data.append({
            "Video": video_name,
            "KTS Segments": len(model_segments),
            "Selected": len(selected_model_segments),
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max()
        })

        del video_feats, recon
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ TVSum results saved to: {EXCEL_OUT}")


# BILSTM - SOFT ATTENTION

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, backend as K

# =======================
# 1️⃣ GPU Memory Optimization
# =======================
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU Memory Growth Enabled")
    except RuntimeError as e:
        print(e)

# Enable mixed precision
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# =======================
# 2️⃣ Load Features with Sliding Windows
# =======================
base_path = "/mnt/d/Mass Projects/Video_Summ/new_dataset_features_npy_1750"
train_path = os.path.join(base_path, "train")
valid_path = os.path.join(base_path, "valid")

SEQ_LEN = 150
STRIDE = 125
FEATURE_DIM = 2048

def load_features(directory):
    all_windows = []
    video_files = sorted(os.listdir(directory))
    for file in video_files:
        file_path = os.path.join(directory, file)
        features = np.load(file_path)
        for start in range(0, len(features) - SEQ_LEN + 1, STRIDE):
            window = features[start:start + SEQ_LEN]
            all_windows.append(window)
    return np.array(all_windows, dtype=np.float32)

print("📥 Loading dataset...")
train_features = load_features(train_path)
valid_features = load_features(valid_path)
print("Train windows:", train_features.shape)
print("Valid windows:", valid_features.shape)

NUM_EPOCHS = 50
BATCH_SIZE = 1
LEARNING_RATE = 1e-5
MODEL_SAVE_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Models_2/SUMME/bilstm_soft_attention_15k.keras"
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)

# =======================
# 3️⃣ Define Soft Attention Layer
# =======================
class SoftAttention(layers.Layer):
    def __init__(self):
        super(SoftAttention, self).__init__()
        self.W = layers.Dense(1, activation='tanh')

    def call(self, inputs):
        # inputs: (batch, time, features)
        score = self.W(inputs)  # (batch, time, 1)
        attention_weights = tf.nn.softmax(score, axis=1)  # normalize over time
        context = attention_weights * inputs  # (batch, time, features)
        context = tf.reduce_sum(context, axis=1, keepdims=True)  # (batch, 1, features)
        # Repeat context to match sequence length
        context_repeated = tf.tile(context, [1, tf.shape(inputs)[1], 1])
        return context_repeated

# =======================
# 4️⃣ Define BiLSTM + Soft Attention Model
# =======================
class BiLSTMSoftAttentionModel(models.Model):
    def __init__(self, feature_dim=2048, lstm_units=64, **kwargs):
        super(BiLSTMSoftAttentionModel, self).__init__(**kwargs)
        self.bilstm = layers.Bidirectional(
            layers.LSTM(lstm_units, return_sequences=True)
        )
        self.attention = SoftAttention()
        self.reconstruction_layer = layers.TimeDistributed(
            layers.Dense(feature_dim, activation="linear")
        )

    def call(self, inputs):
        x = self.bilstm(inputs)
        context = self.attention(x)
        x = x + context  # add context as residual
        reconstructed = self.reconstruction_layer(x)
        return reconstructed

# =======================
# 5️⃣ Build + Compile Model
# =======================
model = BiLSTMSoftAttentionModel(feature_dim=FEATURE_DIM)
model.build(input_shape=(None, SEQ_LEN, FEATURE_DIM))

model.compile(
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
    loss="mse",
    metrics=["mae"]
)

# =======================
# 6️⃣ Training
# =======================
print("🚀 Starting Training...")
history = model.fit(
    train_features, train_features,
    validation_data=(valid_features, valid_features),
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE
)

model.save(MODEL_SAVE_PATH)
print(f"✅ Model saved at {MODEL_SAVE_PATH}")

# =======================
# 7️⃣ Evaluation
# =======================
with tf.keras.utils.custom_object_scope({
    "BiLSTMSoftAttentionModel": BiLSTMSoftAttentionModel,
    "SoftAttention": SoftAttention
}):
    model = tf.keras.models.load_model(MODEL_SAVE_PATH)

print("🔄 Model loaded for evaluation.")

test_video = valid_features[0:1]
reconstructed_video = model.predict(test_video)
print("Reconstructed Features Shape:", reconstructed_video.shape)


# SOFT Attention - SUMME

In [ ]:
import os
import numpy as np
import tensorflow as tf
import scipy.io
import ruptures as rpt
import pandas as pd
import gc

# ========= 1️⃣ Define Soft Attention Model (same as training) =========
class SoftAttention(tf.keras.layers.Layer):
    def __init__(self):
        super(SoftAttention, self).__init__()
        self.W = tf.keras.layers.Dense(1, activation='tanh')

    def call(self, inputs):
        score = self.W(inputs)  # (batch, time, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        context = attention_weights * inputs
        context = tf.reduce_sum(context, axis=1, keepdims=True)
        context_repeated = tf.tile(context, [1, tf.shape(inputs)[1], 1])
        return context_repeated

class BiLSTMSoftAttentionModel(tf.keras.Model):
    def __init__(self, feature_dim=2048, lstm_units=64, **kwargs):
        super(BiLSTMSoftAttentionModel, self).__init__(**kwargs)
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(lstm_units, return_sequences=True)
        )
        self.attention = SoftAttention()
        self.reconstruction_layer = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(feature_dim, activation="linear")
        )

    def call(self, inputs):
        x = self.bilstm(inputs)
        context = self.attention(x)
        x = x + context  # residual connection
        reconstructed = self.reconstruction_layer(x)
        return reconstructed

# ========= 2️⃣ Load Trained Model =========
MODEL_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Models_2/SUMME/bilstm_VAE_60k.keras"
with tf.keras.utils.custom_object_scope({
    "BiLSTMSoftAttentionModel": BiLSTMSoftAttentionModel,
    "SoftAttention": SoftAttention
}):
    model = tf.keras.models.load_model(MODEL_PATH)

print("\n✅ Soft Attention Model loaded successfully")

# ========= 3️⃣ Helper Functions =========
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    cps = algo.predict(pen=penalty)
    segments, start = [], 0
    for cp in cps:
        segments.append((start, cp - 1))
        start = cp
    if segments[-1][1] < features.shape[0] - 1:
        segments.append((start, features.shape[0] - 1))
    return segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]

# ========= 4️⃣ Paths and Settings =========
video_list = [
    "Jumps", "Base Jumping", "Air_Force_One", "Bearpark_climbing", "Bike Polo",
    "Bus_in_Rock_Tunnel", "car_over_camera", "Car_railcrossing", "Cockpit_Landing", "Cooking",
    "Eiffel Tower", "Excavators river crossing", "Fire Domino", "Kids_playing_in_leaves",
    "Notre_Dame", "Paintball", "paluma_jump", "playing_ball", "Playing_on_water_slide",
    "Saving dolphines", "Scuba", "St Maarten Landing", "Statue of Liberty", "Valparaiso_Downhill"
]

FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/extracted_features/"
MAT_DIR = "/mnt/d/Mass Projects/Video_Summ/mat/"
CHUNK_SIZE = 500
PENALTY = 500
FPS = 25
SUMMARY_EXCEL = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Results_2/SUMME/bilstm_soft_attention_35K.xlsx"

# ========= 5️⃣ Evaluate Videos =========
with pd.ExcelWriter(SUMMARY_EXCEL, engine="xlsxwriter") as writer:
    summary_data = []

    for VIDEO_NAME in video_list:
        print(f"\n==============================\n🚀 Processing video: {VIDEO_NAME}\n==============================")

        feats_path = os.path.join(FEATURES_DIR, f"{VIDEO_NAME}.npy")
        mat_path = os.path.join(MAT_DIR, f"{VIDEO_NAME}.mat")
        if not os.path.exists(feats_path) or not os.path.exists(mat_path):
            print("❌ Missing files. Skipping...")
            continue

        raw_feats = np.load(feats_path)
        num_frames = raw_feats.shape[0]
        video_feats = raw_feats[np.newaxis, ...]

        recon = np.zeros_like(video_feats)
        for i in range((num_frames + CHUNK_SIZE - 1) // CHUNK_SIZE):
            s, e = i * CHUNK_SIZE, min((i + 1) * CHUNK_SIZE, num_frames)
            recon[:, s:e, :] = model.predict(video_feats[:, s:e, :], verbose=0)

        print("✅ Features reconstructed")

        errs = np.mean(np.abs(video_feats - recon), axis=2).flatten()
        imp_scores = (errs - errs.min()) / (errs.max() - errs.min())

        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        mat = scipy.io.loadmat(mat_path)
        user_scores = mat['user_score']
        num_users = user_scores.shape[1]
        all_user_metrics = []

        for u in range(num_users):
            usr = user_scores[:, u]
            nonzero_indices = np.where(usr > 0)[0]
            user_segments = get_segments_from_indices(nonzero_indices.tolist())

            f1, precision, recall = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=VIDEO_NAME[:31], index=False)

        summary_data.append({
            "Video": VIDEO_NAME,
            "KTS Segments": len(model_segments),
            "Selected": len(selected_model_segments),
            "Penalty": PENALTY,
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max(),
        })

        del video_feats, recon
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ All results saved to: {SUMMARY_EXCEL}")


# SOFT Attention - TVSUM

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
import gc
import ruptures as rpt

# ========= 1️⃣ Register Custom Classes =========
@tf.keras.utils.register_keras_serializable()
class SoftAttention(tf.keras.layers.Layer):
    def __init__(self):
        super(SoftAttention, self).__init__()
        self.W = tf.keras.layers.Dense(1, activation='tanh')

    def call(self, inputs):
        score = self.W(inputs)  # (batch, time, 1)
        attn_weights = tf.nn.softmax(score, axis=1)  # normalize across time
        context = attn_weights * inputs
        context = tf.reduce_sum(context, axis=1, keepdims=True)  # (batch, 1, features)
        context_repeated = tf.tile(context, [1, tf.shape(inputs)[1], 1])
        return context_repeated


@tf.keras.utils.register_keras_serializable()
class BiLSTMSoftAttentionModel(tf.keras.Model):
    def __init__(self, feature_dim=2048, lstm_units=64, **kwargs):
        super(BiLSTMSoftAttentionModel, self).__init__(**kwargs)
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(lstm_units, return_sequences=True)
        )
        self.attention = SoftAttention()
        self.reconstruction_layer = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(feature_dim, activation="linear")
        )

    def call(self, inputs):
        x = self.bilstm(inputs)
        context = self.attention(x)
        x = x + context  # residual addition
        reconstructed = self.reconstruction_layer(x)
        return reconstructed


# ========= 2️⃣ Load Model =========
MODEL_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/bilstm_soft_attention_15k.keras"

with tf.keras.utils.custom_object_scope({
    "BiLSTMSoftAttentionModel": BiLSTMSoftAttentionModel,
    "SoftAttention": SoftAttention
}):
    model = tf.keras.models.load_model(MODEL_PATH)

print("\n✅ BiLSTM + Soft Attention Model loaded successfully")

# ========= 3️⃣ Helper Functions =========
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    change_points = algo.predict(pen=penalty)
    change_points = [int(cp) for cp in change_points]
    scene_segments, start_frame = [], 0
    for cp in change_points:
        scene_segments.append((start_frame, cp - 1))
        start_frame = cp
    if scene_segments[-1][1] < features.shape[0] - 1:
        scene_segments.append((start_frame, features.shape[0] - 1))
    return scene_segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]

# ========= 4️⃣ TVSum Dataset Paths =========
FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/TVSUM/RandomSplit_Dataset/resnet_npy/extracted_features_25fps_videos"
MAT_FILE_PATH = "/mnt/d/Mass Projects/Video_Summ/ydata-tvsum50-v1_1/ydata-tvsum50-matlab/matlab/ydata-tvsum50.mat"
EXCEL_OUT = "/mnt/d/Mass Projects/Video_Summ/RESULTS/RESULTS/TVSUM/bilstm_soft_attention_15k.xlsx"
FPS = 25
PENALTY = 500
CHUNK_SIZE = 500

# ========= 5️⃣ Load TVSum .mat File =========
with h5py.File(MAT_FILE_PATH, "r") as mat_data:
    tvsum50_group = mat_data["tvsum50"]

    def decode_str(ref):
        return "".join(map(chr, mat_data[ref][()].flatten())).replace("\x00", "")

    video_names = [decode_str(ref) for ref in tvsum50_group["video"][:, 0]]
    user_annos = [mat_data[ref][:] for ref in tvsum50_group["user_anno"][:, 0]]
    nframes_list = [int(mat_data[ref][()][0, 0]) for ref in tvsum50_group["nframes"][:, 0]]

# ========= 6️⃣ Evaluation Loop =========
with pd.ExcelWriter(EXCEL_OUT, engine="xlsxwriter") as writer:
    summary_data = []

    for idx, video_name in enumerate(video_names):
        print(f"\n🚀 Evaluating video: {video_name}")

        feature_path = os.path.join(FEATURES_DIR, f"{video_name}.npy")
        if not os.path.exists(feature_path):
            print(f"❌ Missing feature file: {video_name}")
            continue

        raw_feats = np.load(feature_path)
        num_frames = raw_feats.shape[0]
        video_feats = raw_feats[np.newaxis, ...]

        # Reconstruct in chunks
        recon = np.zeros_like(video_feats)
        for i in range((num_frames + CHUNK_SIZE - 1) // CHUNK_SIZE):
            s, e = i * CHUNK_SIZE, min((i + 1) * CHUNK_SIZE, num_frames)
            recon[:, s:e, :] = model.predict(video_feats[:, s:e, :], verbose=0)

        # Compute importance
        errs = np.mean(np.abs(video_feats - recon), axis=2).flatten()
        imp_scores = (errs - errs.min()) / (errs.max() - errs.min())

        # KTS segmentation + knapsack selection
        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        # Evaluate per-user
        user_scores = user_annos[idx]
        num_users = user_scores.shape[0]
        all_user_metrics = []

        for u in range(num_users):
            user_score = user_scores[u]
            k = int(np.floor(num_frames * 0.15))
            selected_indices = np.argsort(user_score)[-k:]
            user_segments = get_segments_from_indices(selected_indices.tolist())

            f1, precision, recall = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=video_name[:31], index=False)

        summary_data.append({
            "Video": video_name,
            "KTS Segments": len(model_segments),
            "Selected": len(selected_model_segments),
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max()
        })

        del video_feats, recon
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ TVSum results saved to: {EXCEL_OUT}")


# VAE BILSTM

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# =======================
# 1️⃣ GPU Memory Optimization
# =======================
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU Memory Growth Enabled")
    except RuntimeError as e:
        print(e)

# Enable mixed precision
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# =======================
# 2️⃣ Load Features with Sliding Windows
# =======================
base_path = "/mnt/d/Mass Projects/Video_Summ/new_dataset_features_npy_1750"
train_path = os.path.join(base_path, "train")
valid_path = os.path.join(base_path, "valid")

SEQ_LEN = 150
STRIDE = 125
FEATURE_DIM = 2048

def load_features(directory):
    all_windows = []
    video_files = sorted(os.listdir(directory))
    for file in video_files:
        file_path = os.path.join(directory, file)
        features = np.load(file_path)  # (1750, 2048)

        # Sliding windows
        for start in range(0, len(features) - SEQ_LEN + 1, STRIDE):
            window = features[start:start+SEQ_LEN]
            all_windows.append(window)
    return np.array(all_windows, dtype=np.float32)

print("📥 Loading dataset...")
train_features = load_features(train_path)
valid_features = load_features(valid_path)
print("Train windows:", train_features.shape)
print("Valid windows:", valid_features.shape)

NUM_EPOCHS = 50
BATCH_SIZE = 1
LEARNING_RATE = 1e-4
MODEL_SAVE_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Models_2/SUMME/bilstm_VAE_30k.keras"
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)

# =======================
# 3️⃣ Define BiLSTM + Attention VAE Model
# =======================
class BiLSTMAttentionVAE(models.Model):
    def __init__(self, feature_dim, latent_dim=128, num_heads=2, lstm_units=64, **kwargs):
        super(BiLSTMAttentionVAE, self).__init__(**kwargs)
        self.feature_dim = feature_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.lstm_units = lstm_units

        # Encoder
        self.bilstm = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True))
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim // num_heads)

        # Latent projection
        self.mu = layers.Dense(latent_dim)
        self.logvar = layers.Dense(latent_dim)

        # Decoder
        self.decoder = layers.TimeDistributed(layers.Dense(feature_dim, activation="linear"))

    def reparameterize(self, mu, logvar):
        eps = tf.random.normal(shape=tf.shape(mu), dtype=mu.dtype)
        return mu + tf.exp(0.5 * logvar) * eps

    def call(self, inputs):
        h = self.bilstm(inputs)
        h = self.attention(h, h)

        mu, logvar = self.mu(h), self.logvar(h)
        z = self.reparameterize(mu, logvar)

        recon = self.decoder(z)
        return recon, mu, logvar

    # 🔧 Add this so Keras can save/load properly
    def get_config(self):
        config = super().get_config()
        config.update({
            "feature_dim": self.feature_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
            "lstm_units": self.lstm_units,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


# =======================
# 4️⃣ Custom Loss (Reconstruction + KL Divergence)
# =======================
@tf.function
def vae_loss(y_true, y_pred, mu, logvar):
    # Force float32 for stability
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    mu = tf.cast(mu, tf.float32)
    logvar = tf.cast(logvar, tf.float32)

    # Reconstruction loss
    recon_loss = tf.reduce_mean(tf.square(y_true - y_pred))

    # KL divergence
    kl_loss = -0.5 * tf.reduce_mean(1 + logvar - tf.square(mu) - tf.exp(logvar))

    return recon_loss + 0.01 * kl_loss, recon_loss, kl_loss


# =======================
# 5️⃣ Training Loop (Custom, not model.fit)
# =======================
model = BiLSTMAttentionVAE(feature_dim=FEATURE_DIM)

optimizer = optimizers.Adam(learning_rate=LEARNING_RATE)

train_dataset = tf.data.Dataset.from_tensor_slices(train_features).batch(BATCH_SIZE).shuffle(100)
valid_dataset = tf.data.Dataset.from_tensor_slices(valid_features).batch(BATCH_SIZE)

for epoch in range(NUM_EPOCHS):
    # Training
    train_losses = []
    for batch in train_dataset:
        with tf.GradientTape() as tape:
            recon, mu, logvar = model(batch)
            loss, recon_loss, kl_loss = vae_loss(batch, recon, mu, logvar)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_losses.append(loss.numpy())

    # Validation
    val_losses = []
    for batch in valid_dataset:
        recon, mu, logvar = model(batch)
        loss, _, _ = vae_loss(batch, recon, mu, logvar)
        val_losses.append(loss.numpy())

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train Loss: {np.mean(train_losses):.4f} | Val Loss: {np.mean(val_losses):.4f}")

# =======================
# 6️⃣ Save & Reload
# =======================
model.save(MODEL_SAVE_PATH)
print(f"✅ Model saved at {MODEL_SAVE_PATH}")

with tf.keras.utils.custom_object_scope({"BiLSTMAttentionVAE": BiLSTMAttentionVAE}):
    model = tf.keras.models.load_model(MODEL_SAVE_PATH)

print("🔄 Model loaded for evaluation.")

# Example Evaluation
test_video = valid_features[0:1]
recon, mu, logvar = model(test_video)
print("Reconstructed Features Shape:", recon.shape)


# VAE - SUMME

In [ ]:
import os
import numpy as np
import tensorflow as tf
import scipy.io
import ruptures as rpt
import pandas as pd
import gc

# ========= 1️⃣ Define Trained Model Class (same as training) =========
SEQ_LEN = 150
FEATURE_DIM = 2048

class BiLSTM_VAE_Attention(tf.keras.Model):
    def __init__(self, feature_dim, latent_dim=256, lstm_units=64, num_heads=2, **kwargs):
        super(BiLSTM_VAE_Attention, self).__init__(**kwargs)
        self.feature_dim = feature_dim
        self.latent_dim = latent_dim
        self.lstm_units = lstm_units
        self.num_heads = num_heads

        self.bilstm_enc = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(lstm_units, return_sequences=True, dtype="float32")
        )
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=feature_dim // num_heads, dtype="float32"
        )
        self.flatten = tf.keras.layers.Flatten(dtype="float32")
        self.mu_dense = tf.keras.layers.Dense(latent_dim, dtype="float32")
        self.logvar_dense = tf.keras.layers.Dense(latent_dim, dtype="float32")

        self.decoder_dense = tf.keras.layers.Dense(SEQ_LEN * feature_dim, activation="linear", dtype="float32")
        self.reshape_layer = tf.keras.layers.Reshape((SEQ_LEN, feature_dim))

    def reparameterize(self, mu, logvar):
        eps = tf.random.normal(shape=tf.shape(mu), dtype=mu.dtype)
        return mu + tf.exp(0.5 * logvar) * eps

    def call(self, inputs):
        x = tf.cast(inputs, tf.float32)
        x = self.bilstm_enc(x)
        x = self.attention(x, x)
        flat = self.flatten(x)
        mu = self.mu_dense(flat)
        logvar = self.logvar_dense(flat)
        z = self.reparameterize(mu, logvar)
        decoded = self.decoder_dense(z)
        reconstructed = self.reshape_layer(decoded)
        kl_loss = -0.5 * tf.reduce_mean(1 + logvar - tf.square(mu) - tf.exp(logvar))
        self.add_loss(kl_loss)
        return reconstructed


# ========= 2️⃣ Load Trained Model =========
MODEL_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/bilstm_VAE_60k.keras"

with tf.keras.utils.custom_object_scope({"BiLSTM_VAE_Attention": BiLSTM_VAE_Attention}):
    model = tf.keras.models.load_model(MODEL_PATH)

print("\n✅ BiLSTM + Multi-Head Attention + VAE model loaded successfully")

# ========= 3️⃣ Helper Functions =========
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    cps = algo.predict(pen=penalty)
    segments, start = [], 0
    for cp in cps:
        segments.append((start, cp - 1))
        start = cp
    if segments[-1][1] < features.shape[0] - 1:
        segments.append((start, features.shape[0] - 1))
    return segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]


# ========= 4️⃣ Paths and Settings =========
video_list = [
    "Jumps", "Base Jumping", "Air_Force_One", "Bearpark_climbing", "Bike Polo",
    "Bus_in_Rock_Tunnel", "car_over_camera", "Car_railcrossing", "Cockpit_Landing", "Cooking",
    "Eiffel Tower", "Excavators river crossing", "Fire Domino", "Kids_playing_in_leaves",
    "Notre_Dame", "Paintball", "paluma_jump", "playing_ball", "Playing_on_water_slide",
    "Saving dolphines", "Scuba", "St Maarten Landing", "Statue of Liberty", "Valparaiso_Downhill"
]

FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/extracted_features/"
MAT_DIR = "/mnt/d/Mass Projects/Video_Summ/mat/"
CHUNK_SIZE = 500
PENALTY = 500
FPS = 25
SUMMARY_EXCEL = "/mnt/d/Mass Projects/Video_Summ/RESULTS/RESULTS/SUMME/bilstm_VAE_60k.xlsx"


# ========= 5️⃣ Evaluate Videos =========
with pd.ExcelWriter(SUMMARY_EXCEL, engine="xlsxwriter") as writer:
    summary_data = []

    for VIDEO_NAME in video_list:
        print(f"\n==============================\n🚀 Processing video: {VIDEO_NAME}\n==============================")

        feats_path = os.path.join(FEATURES_DIR, f"{VIDEO_NAME}.npy")
        mat_path = os.path.join(MAT_DIR, f"{VIDEO_NAME}.mat")
        if not os.path.exists(feats_path) or not os.path.exists(mat_path):
            print("❌ Missing files. Skipping...")
            continue

        raw_feats = np.load(feats_path)
        num_frames = raw_feats.shape[0]
        video_feats = raw_feats[np.newaxis, ...]

        recon = np.zeros_like(video_feats)
        for i in range((num_frames + CHUNK_SIZE - 1) // CHUNK_SIZE):
            s, e = i * CHUNK_SIZE, min((i + 1) * CHUNK_SIZE, num_frames)
            recon[:, s:e, :] = model.predict(video_feats[:, s:e, :], verbose=0)

        print("✅ Features reconstructed")

        errs = np.mean(np.abs(video_feats - recon), axis=2).flatten()
        imp_scores = (errs - errs.min()) / (errs.max() - errs.min())

        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        mat = scipy.io.loadmat(mat_path)
        user_scores = mat['user_score']
        num_users = user_scores.shape[1]
        all_user_metrics = []

        for u in range(num_users):
            usr = user_scores[:, u]
            nonzero_indices = np.where(usr > 0)[0]
            user_segments = get_segments_from_indices(nonzero_indices.tolist())
            f1, precision, recall = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=VIDEO_NAME[:31], index=False)

        summary_data.append({
            "Video": VIDEO_NAME,
            "KTS Segments": len(model_segments),
            "Selected": len(selected_model_segments),
            "Penalty": PENALTY,
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max(),
        })

        del video_feats, recon
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ All results saved to: {SUMMARY_EXCEL}")


In [ ]:
import os
import numpy as np
import tensorflow as tf
import scipy.io
import ruptures as rpt
import pandas as pd
import gc

# ========= 1. Load Model (VAE) =========
class BiLSTMAttentionVAE(tf.keras.Model):
    def __init__(self, feature_dim=2048, latent_dim=128, num_heads=2, lstm_units=64, **kwargs):
        super(BiLSTMAttentionVAE, self).__init__(**kwargs)
        self.feature_dim = feature_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.lstm_units = lstm_units

        # Encoder
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(lstm_units, return_sequences=True))
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=feature_dim // num_heads)

        # Latent space
        self.mu = tf.keras.layers.Dense(latent_dim)
        self.logvar = tf.keras.layers.Dense(latent_dim)

        # Decoder
        self.decoder = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(feature_dim, activation="linear"))

    def reparameterize(self, mu, logvar):
        eps = tf.random.normal(shape=tf.shape(mu), dtype=mu.dtype)
        return mu + tf.exp(0.5 * logvar) * eps

    def call(self, inputs):
        h = self.bilstm(inputs)
        h = self.attention(h, h)
        mu, logvar = self.mu(h), self.logvar(h)
        z = self.reparameterize(mu, logvar)
        recon = self.decoder(z)
        return recon, mu, logvar

    def get_config(self):
        config = super().get_config()
        config.update({
            "feature_dim": self.feature_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
            "lstm_units": self.lstm_units,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


MODEL_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/bilstm_VAE_35k.keras"
with tf.keras.utils.custom_object_scope({"BiLSTMAttentionVAE": BiLSTMAttentionVAE}):
    model = tf.keras.models.load_model(MODEL_PATH)

print("\n✅ VAE model loaded successfully")

# ========= 2. Helper Functions (unchanged) =========
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def format_segments(segments):
    return [f"({s})" if s == e else f"({s}–{e})" for s, e in segments]

def print_segments_columnwise(segments, items_per_line=10):
    for i in range(0, len(segments), items_per_line):
        print("  " + "  ".join(segments[i:i + items_per_line]))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    matched = []
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                matched.append(m_seg)
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall, matched

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    change_points = algo.predict(pen=penalty)
    change_points = [int(cp) for cp in change_points]
    scene_segments = []
    start_frame = 0
    for cp in change_points:
        scene_segments.append((start_frame, cp - 1))
        start_frame = cp
    if scene_segments[-1][1] < features.shape[0] - 1:
        scene_segments.append((start_frame, features.shape[0] - 1))
    return scene_segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]

# ========= 3. Process Videos =========
video_list = [
    "Jumps", "Base Jumping", "Air_Force_One", "Bearpark_climbing", "Bike Polo",
    "Bus_in_Rock_Tunnel", "car_over_camera", "Car_railcrossing", "Cockpit_Landing", "Cooking",
    "Eiffel Tower", "Excavators river crossing", "Fire Domino", "Kids_playing_in_leaves",
    "Notre_Dame", "Paintball", "paluma_jump", "playing_ball", "Playing_on_water_slide",
    "Saving dolphines", "Scuba", "St Maarten Landing", "Statue of Liberty", "Valparaiso_Downhill"
]

FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/extracted_features/"
MAT_DIR = "/mnt/d/Mass Projects/Video_Summ/mat/"
CHUNK_SIZE = 500
PENALTY = 500
FPS = 25
SUMMARY_EXCEL = "/mnt/d/Mass Projects/Video_Summ/RESULTS/RESULTS/SUMME/bilstm_VAE_35k.xlsx"

with pd.ExcelWriter(SUMMARY_EXCEL, engine="xlsxwriter") as writer:
    summary_data = []

    for VIDEO_NAME in video_list:
        print(f"\n==============================\n🚀 Processing video: {VIDEO_NAME}\n==============================")

        feats_path = os.path.join(FEATURES_DIR, f"{VIDEO_NAME}.npy")
        mat_path = os.path.join(MAT_DIR, f"{VIDEO_NAME}.mat")
        if not os.path.exists(feats_path) or not os.path.exists(mat_path):
            print("❌ Skipping, missing file.")
            continue

        raw_feats = np.load(feats_path)
        num_frames = raw_feats.shape[0]
        video_feats = raw_feats[np.newaxis, ...]

        recon = np.zeros_like(video_feats)
        for i in range((num_frames + CHUNK_SIZE - 1) // CHUNK_SIZE):
            s, e = i * CHUNK_SIZE, min((i + 1) * CHUNK_SIZE, num_frames)
            # call VAE -> take only reconstruction
            r, _, _ = model(video_feats[:, s:e, :])
            recon[:, s:e, :] = r.numpy()
        print("✅ Features reconstructed (VAE)")

        # === scoring & segmentation remain unchanged ===
        errs = np.mean(np.abs(video_feats - recon), axis=2).flatten()
        imp_scores = (errs - errs.min()) / (errs.max() - errs.min())

        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        mat = scipy.io.loadmat(mat_path)
        user_scores = mat['user_score']
        num_users = user_scores.shape[1]
        all_user_metrics = []

        for u in range(num_users):
            usr = user_scores[:, u]
            nonzero_indices = np.where(usr > 0)[0]
            user_segments = get_segments_from_indices(nonzero_indices.tolist())

            f1, precision, recall, _ = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=VIDEO_NAME[:31], index=False)

        summary_data.append({
            "Video Name": VIDEO_NAME,
            "KTS Segments": len(model_segments),
            "Knapsack Selected": len(selected_model_segments),
            "Penalty": PENALTY,
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max(),
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
        })

        del video_feats, recon
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ All results saved to: {SUMMARY_EXCEL}")


# VAE - TVSUM

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
import ruptures as rpt
import gc
import scipy.io

# ========= 1️⃣ Define & Load VAE Model =========
class BiLSTMAttentionVAE(tf.keras.Model):
    def __init__(self, feature_dim=2048, latent_dim=128, num_heads=2, lstm_units=64, **kwargs):
        super(BiLSTMAttentionVAE, self).__init__(**kwargs)
        self.feature_dim = feature_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.lstm_units = lstm_units

        # Encoder
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(lstm_units, return_sequences=True)
        )
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=feature_dim // num_heads
        )

        # Latent space
        self.mu = tf.keras.layers.Dense(latent_dim)
        self.logvar = tf.keras.layers.Dense(latent_dim)

        # Decoder
        self.decoder = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(feature_dim, activation="linear")
        )

    def reparameterize(self, mu, logvar):
        eps = tf.random.normal(shape=tf.shape(mu), dtype=mu.dtype)
        return mu + tf.exp(0.5 * logvar) * eps

    def call(self, inputs):
        h = self.bilstm(inputs)
        h = self.attention(h, h)
        mu, logvar = self.mu(h), self.logvar(h)
        z = self.reparameterize(mu, logvar)
        recon = self.decoder(z)
        return recon, mu, logvar

    def get_config(self):
        config = super().get_config()
        config.update({
            "feature_dim": self.feature_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
            "lstm_units": self.lstm_units,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


MODEL_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Models_2/SUMME/bilstm_VAE_60k.keras"
with tf.keras.utils.custom_object_scope({"BiLSTMAttentionVAE": BiLSTMAttentionVAE}):
    model = tf.keras.models.load_model(MODEL_PATH)
print("\n✅ VAE model loaded successfully for TVSum evaluation")

# ========= 2️⃣ Helper Functions =========
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    change_points = algo.predict(pen=penalty)
    change_points = [int(cp) for cp in change_points]
    segments, start = [], 0
    for cp in change_points:
        segments.append((start, cp - 1))
        start = cp
    if segments[-1][1] < features.shape[0] - 1:
        segments.append((start, features.shape[0] - 1))
    return segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]

# ========= 3️⃣ Dataset Paths =========
FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/TVSUM/RandomSplit_Dataset/resnet_npy/extracted_features_25fps_videos"
MAT_FILE_PATH = "/mnt/d/Mass Projects/Video_Summ/ydata-tvsum50-v1_1/ydata-tvsum50-matlab/matlab/ydata-tvsum50.mat"
EXCEL_OUT = "/mnt/d/Mass Projects/Video_Summ/RESULTS/Results_2/TVSUM/bilstm_VAE_60k.xlsx"

FPS = 25
PENALTY = 500
CHUNK_SIZE = 500

# ========= 4️⃣ Load TVSum .mat File =========
with h5py.File(MAT_FILE_PATH, "r") as mat_data:
    tvsum50 = mat_data["tvsum50"]

    def decode_str(ref):
        return "".join(map(chr, mat_data[ref][()].flatten())).replace("\x00", "")

    video_names = [decode_str(ref) for ref in tvsum50["video"][:, 0]]
    user_annos = [mat_data[ref][:] for ref in tvsum50["user_anno"][:, 0]]
    nframes_list = [int(mat_data[ref][()][0, 0]) for ref in tvsum50["nframes"][:, 0]]

# ========= 5️⃣ Evaluation =========
with pd.ExcelWriter(EXCEL_OUT, engine="xlsxwriter") as writer:
    summary_data = []

    for idx, video_name in enumerate(video_names):
        print(f"\n🚀 Processing video: {video_name}")

        feature_path = os.path.join(FEATURES_DIR, f"{video_name}.npy")
        if not os.path.exists(feature_path):
            print(f"❌ Missing feature file for {video_name}, skipping...")
            continue

        raw_feats = np.load(feature_path)
        num_frames = raw_feats.shape[0]
        video_feats = raw_feats[np.newaxis, ...]

        # === Reconstruction ===
        recon = np.zeros_like(video_feats)
        for i in range((num_frames + CHUNK_SIZE - 1) // CHUNK_SIZE):
            s, e = i * CHUNK_SIZE, min((i + 1) * CHUNK_SIZE, num_frames)
            r, _, _ = model(video_feats[:, s:e, :])
            recon[:, s:e, :] = r.numpy()
        print("✅ Features reconstructed")

        # === Importance Scoring ===
        errs = np.mean(np.abs(video_feats - recon), axis=2).flatten()
        imp_scores = (errs - errs.min()) / (errs.max() - errs.min())

        # === KTS + Knapsack ===
        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        # === Per-user evaluation ===
        user_scores = user_annos[idx]
        num_users = user_scores.shape[0]
        all_user_metrics = []

        for u in range(num_users):
            user_score = user_scores[u]
            k = int(np.floor(num_frames * 0.15))
            top_indices = np.argsort(user_score)[-k:]
            user_segments = get_segments_from_indices(top_indices.tolist())

            f1, precision, recall = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=video_name[:31], index=False)

        summary_data.append({
            "Video": video_name,
            "KTS Segments": len(model_segments),
            "Knapsack Selected": len(selected_model_segments),
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max()
        })

        del video_feats, recon
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ All TVSum results saved to: {EXCEL_OUT}")


# SELECTOR

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt

# =======================
# 1️⃣ GPU Optimization
# =======================
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(e)

tf.keras.mixed_precision.set_global_policy("mixed_float16")

# =======================
# 2️⃣ Dataset Loading (train/valid/test)
# =======================
BASE_PATH = "/mnt/d/Mass Projects/Video_Summ/new_dataset_features_npy_1750"
SEQ_LEN = 150
STRIDE = 125
FEATURE_DIM = 2048

def load_features_from_folder(folder_path):
    all_windows = []
    for fname in sorted(os.listdir(folder_path)):
        fpath = os.path.join(folder_path, fname)
        if not fname.endswith(".npy"):
            continue
        feats = np.load(fpath)
        for start in range(0, len(feats) - SEQ_LEN + 1, STRIDE):
            window = feats[start:start+SEQ_LEN]
            all_windows.append(window)
    return np.array(all_windows, dtype=np.float32)

print("📥 Loading dataset...")
train_features = load_features_from_folder(os.path.join(BASE_PATH, "train"))
valid_features = load_features_from_folder(os.path.join(BASE_PATH, "valid"))
test_features = load_features_from_folder(os.path.join(BASE_PATH, "test"))

print("Train:", train_features.shape)
print("Valid:", valid_features.shape)
print("Test:", test_features.shape)

# =======================
# 3️⃣ Reconstructor Model
# =======================
class Reconstructor(tf.keras.Model):
    def __init__(self, input_dim=2048, hidden_dim=512):
        super(Reconstructor, self).__init__()
        self.encoder = layers.Bidirectional(
            layers.LSTM(hidden_dim, return_sequences=True)
        )
        self.decoder = layers.Bidirectional(
            layers.LSTM(hidden_dim, return_sequences=True)
        )
        self.output_layer = layers.TimeDistributed(
            layers.Dense(input_dim, activation='linear')
        )

    def call(self, inputs):
        x = self.encoder(inputs)
        x = self.decoder(x)
        return self.output_layer(x)

def reconstruction_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

reconstructor = Reconstructor(input_dim=FEATURE_DIM)
reconstructor.compile(optimizer=optimizers.Adam(1e-4), loss=reconstruction_loss)

print("🚀 Training Reconstructor...")
recon_history = reconstructor.fit(
    train_features, train_features,
    validation_data=(valid_features, valid_features),
    epochs=50, batch_size=2
)

RECON_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/reconstructor_model_35k.keras"
reconstructor.save(RECON_PATH)
print(f"✅ Reconstructor saved at {RECON_PATH}")

# =======================
# 4️⃣ Generate Reconstruction-Based Importance Labels
# =======================
print("⚙️ Generating importance labels...")
train_recon = reconstructor.predict(train_features)
valid_recon = reconstructor.predict(valid_features)

train_err = np.mean(np.square(train_features - train_recon), axis=-1, keepdims=True)
valid_err = np.mean(np.square(valid_features - valid_recon), axis=-1, keepdims=True)

# Normalize 0–1
train_labels = (train_err - np.min(train_err)) / (np.max(train_err) - np.min(train_err))
valid_labels = (valid_err - np.min(valid_err)) / (np.max(valid_err) - np.min(valid_err))

print("Train Labels:", train_labels.shape, "Valid Labels:", valid_labels.shape)

# =======================
# 5️⃣ Selector Model
# =======================
class Selector(tf.keras.Model):
    def __init__(self, input_dim=2048, hidden_dim=512):
        super(Selector, self).__init__()
        self.bilstm = layers.Bidirectional(
            layers.LSTM(hidden_dim, return_sequences=True)
        )
        self.output_layer = layers.TimeDistributed(
            layers.Dense(1, activation='sigmoid')
        )

    def call(self, inputs):
        x = self.bilstm(inputs)
        return self.output_layer(x)

def sparsity_loss(y_true, y_pred):
    sigma = 0.7
    return tf.abs(tf.reduce_mean(y_pred) - sigma)

selector = Selector(input_dim=FEATURE_DIM)
selector.compile(optimizer=optimizers.Adam(1e-4), loss=sparsity_loss)

print("🚀 Training Selector...")
sel_history = selector.fit(
    train_features, train_labels,
    validation_data=(valid_features, valid_labels),
    epochs=50, batch_size=2
)

SELECTOR_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/selector_model_35k.keras"
selector.save(SELECTOR_PATH)
print(f"✅ Selector saved at {SELECTOR_PATH}")

# =======================
# 6️⃣ Visualize Losses
# =======================
plt.figure(figsize=(10,5))
plt.plot(recon_history.history['loss'], label='Reconstructor Train Loss')
plt.plot(recon_history.history['val_loss'], label='Reconstructor Val Loss')
plt.legend(); plt.show()

plt.figure(figsize=(10,5))
plt.plot(sel_history.history['loss'], label='Selector Train Loss')
plt.plot(sel_history.history['val_loss'], label='Selector Val Loss')
plt.legend(); plt.show()

# =======================
# 7️⃣ Test Prediction Example
# =======================
print("🔍 Generating importance scores on test sample...")
test_video = test_features[0:1]
scores = selector.predict(test_video).flatten()
print("Predicted importance scores:", scores[:20])


# SELECTOR - SUMME

In [ ]:
import os
import numpy as np
import tensorflow as tf
import scipy.io
import ruptures as rpt
import pandas as pd
import gc

# =====================================
# 1️⃣ Define Selector Model (with get_config fix)
# =====================================
SEQ_LEN = 150
FEATURE_DIM = 2048

class Selector(tf.keras.Model):
    def __init__(self, input_dim=2048, hidden_dim=512, **kwargs):
        super(Selector, self).__init__(**kwargs)
        self.input_dim_ = input_dim
        self.hidden_dim_ = hidden_dim
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(hidden_dim, return_sequences=True, dtype="float32")
        )
        self.output_layer = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(1, activation='sigmoid', dtype="float32")
        )

    def call(self, inputs):
        x = self.bilstm(inputs)
        return self.output_layer(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            "input_dim": self.input_dim_,
            "hidden_dim": self.hidden_dim_,
        })
        return config

# =====================================
# 2️⃣ Load Pretrained Selector
# =====================================
SELECTOR_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/selector_model.keras"

selector = Selector(input_dim=FEATURE_DIM, hidden_dim=512)
# Build once so weights can load
selector.build((None, SEQ_LEN, FEATURE_DIM))

try:
    # Try as weights first
    selector.load_weights(SELECTOR_PATH)
    print("\n✅ Selector weights loaded successfully (via load_weights)")
except Exception as e:
    print(f"\n⚠️ Weight loading failed: {e}")
    print("➡️ Trying to load full .keras model...")
    with tf.keras.utils.custom_object_scope({"Selector": Selector}):
        selector = tf.keras.models.load_model(SELECTOR_PATH)
    print("\n✅ Selector model loaded successfully (via load_model fallback)")

# =====================================
# 3️⃣ Helper Functions
# =====================================
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    cps = algo.predict(pen=penalty)
    segments, start = [], 0
    for cp in cps:
        segments.append((start, cp - 1))
        start = cp
    if segments[-1][1] < features.shape[0] - 1:
        segments.append((start, features.shape[0] - 1))
    return segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]

# =====================================
# 4️⃣ Paths and Settings
# =====================================
video_list = [
    "Jumps", "Base Jumping", "Air_Force_One", "Bearpark_climbing", "Bike Polo",
    "Bus_in_Rock_Tunnel", "car_over_camera", "Car_railcrossing", "Cockpit_Landing", "Cooking",
    "Eiffel Tower", "Excavators river crossing", "Fire Domino", "Kids_playing_in_leaves",
    "Notre_Dame", "Paintball", "paluma_jump", "playing_ball", "Playing_on_water_slide",
    "Saving dolphines", "Scuba", "St Maarten Landing", "Statue of Liberty", "Valparaiso_Downhill"
]

FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/extracted_features/"
MAT_DIR = "/mnt/d/Mass Projects/Video_Summ/mat/"
FPS = 25
PENALTY = 500
SUMMARY_EXCEL = "/mnt/d/Mass Projects/Video_Summ/RESULTS/RESULTS/SUMME/selector_model_eval.xlsx"

# =====================================
# 5️⃣ Evaluate Videos
# =====================================
with pd.ExcelWriter(SUMMARY_EXCEL, engine="xlsxwriter") as writer:
    summary_data = []

    for VIDEO_NAME in video_list:
        print(f"\n==============================\n🚀 Processing video: {VIDEO_NAME}\n==============================")

        feats_path = os.path.join(FEATURES_DIR, f"{VIDEO_NAME}.npy")
        mat_path = os.path.join(MAT_DIR, f"{VIDEO_NAME}.mat")
        if not os.path.exists(feats_path) or not os.path.exists(mat_path):
            print("❌ Missing files. Skipping...")
            continue

        raw_feats = np.load(feats_path)
        num_frames = raw_feats.shape[0]

        seqs = []
        for start in range(0, num_frames - SEQ_LEN + 1, SEQ_LEN):
            seq = raw_feats[start:start + SEQ_LEN]
            seqs.append(seq)
        if not seqs:
            continue
        seqs = np.array(seqs, dtype=np.float32)

        preds = selector.predict(seqs, verbose=0)
        preds = preds.reshape(-1)[:num_frames]

        imp_scores = (preds - preds.min()) / (preds.max() - preds.min() + 1e-8)

        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        mat = scipy.io.loadmat(mat_path)
        user_scores = mat['user_score']
        num_users = user_scores.shape[1]
        all_user_metrics = []

        for u in range(num_users):
            usr = user_scores[:, u]
            nonzero_indices = np.where(usr > 0)[0]
            user_segments = get_segments_from_indices(nonzero_indices.tolist())
            f1, precision, recall = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=VIDEO_NAME[:31], index=False)

        summary_data.append({
            "Video": VIDEO_NAME,
            "KTS Segments": len(model_segments),
            "Selected": len(selected_model_segments),
            "Penalty": PENALTY,
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max(),
        })

        del seqs
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ All results saved to: {SUMMARY_EXCEL}")


# SELECTOR - TVSUM

In [ ]:
import os
import numpy as np
import tensorflow as tf
import scipy.io
import ruptures as rpt
import pandas as pd
import gc

# ===============================
# 1️⃣ Define Selector Model
# ===============================
SEQ_LEN = 150
FEATURE_DIM = 2048

class Selector(tf.keras.Model):
    def __init__(self, input_dim=2048, hidden_dim=512, **kwargs):
        super(Selector, self).__init__(**kwargs)
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(hidden_dim, return_sequences=True, dtype="float32")
        )
        self.output_layer = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(1, activation='sigmoid', dtype="float32")
        )

    def call(self, inputs):
        x = self.bilstm(inputs)
        return self.output_layer(x)


# ===============================
# 2️⃣ Load Trained Selector Weights
# ===============================
SELECTOR_WEIGHTS = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/selector_model.keras"

selector = Selector(input_dim=FEATURE_DIM, hidden_dim=512)
dummy_input = tf.random.normal((1, SEQ_LEN, FEATURE_DIM))
selector(dummy_input)  # build model before loading weights
selector.load_weights(SELECTOR_WEIGHTS)
print("\n✅ Selector weights loaded successfully")

# ===============================
# 3️⃣ Helper Functions
# ===============================
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    cps = algo.predict(pen=penalty)
    segments, start = [], 0
    for cp in cps:
        segments.append((start, cp - 1))
        start = cp
    if segments[-1][1] < features.shape[0] - 1:
        segments.append((start, features.shape[0] - 1))
    return segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]


# ===============================
# 4️⃣ Paths and Settings
# ===============================
video_list = [
    "Jumps", "Base Jumping", "Air_Force_One", "Bearpark_climbing", "Bike Polo",
    "Bus_in_Rock_Tunnel", "car_over_camera", "Car_railcrossing", "Cockpit_Landing", "Cooking",
    "Eiffel Tower", "Excavators river crossing", "Fire Domino", "Kids_playing_in_leaves",
    "Notre_Dame", "Paintball", "paluma_jump", "playing_ball", "Playing_on_water_slide",
    "Saving dolphines", "Scuba", "St Maarten Landing", "Statue of Liberty", "Valparaiso_Downhill"
]

FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/extracted_features/"
MAT_DIR = "/mnt/d/Mass Projects/Video_Summ/mat/"
PENALTY = 500
FPS = 25
SUMMARY_EXCEL = "/mnt/d/Mass Projects/Video_Summ/RESULTS/RESULTS/TVSUM/selector_model_eval.xlsx"

# ===============================
# 5️⃣ Evaluate on All Videos
# ===============================
with pd.ExcelWriter(SUMMARY_EXCEL, engine="xlsxwriter") as writer:
    summary_data = []

    for VIDEO_NAME in video_list:
        print(f"\n==============================\n🚀 Processing video: {VIDEO_NAME}\n==============================")

        feats_path = os.path.join(FEATURES_DIR, f"{VIDEO_NAME}.npy")
        mat_path = os.path.join(MAT_DIR, f"{VIDEO_NAME}.mat")

        if not os.path.exists(feats_path) or not os.path.exists(mat_path):
            print("❌ Missing files. Skipping...")
            continue

        raw_feats = np.load(feats_path)
        num_frames = raw_feats.shape[0]

        # Pad/crop sequences
        seqs = []
        for start in range(0, num_frames - SEQ_LEN + 1, SEQ_LEN):
            seq = raw_feats[start:start + SEQ_LEN]
            seqs.append(seq)
        if not seqs:
            continue
        seqs = np.array(seqs, dtype=np.float32)

        # Predict importance
        preds = selector.predict(seqs, verbose=0)
        preds = preds.reshape(-1)[:num_frames]
        imp_scores = (preds - preds.min()) / (preds.max() - preds.min() + 1e-8)

        # ========= Segmentation + Knapsack =========
        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        # ========= Load User Annotations =========
        mat = scipy.io.loadmat(mat_path)
        user_scores = mat['user_score']
        num_users = user_scores.shape[1]
        all_user_metrics = []

        for u in range(num_users):
            usr = user_scores[:, u]
            nonzero_indices = np.where(usr > 0)[0]
            user_segments = get_segments_from_indices(nonzero_indices.tolist())
            f1, precision, recall = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=VIDEO_NAME[:31], index=False)

        summary_data.append({
            "Video": VIDEO_NAME,
            "KTS Segments": len(model_segments),
            "Selected": len(selected_model_segments),
            "Penalty": PENALTY,
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max(),
        })

        del seqs
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ All results saved to: {SUMMARY_EXCEL}")


In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
import ruptures as rpt
import gc

# ========= 1️⃣ Define and Register Selector & Custom Loss =========
@tf.keras.utils.register_keras_serializable()
def sparsity_loss(y_true, y_pred):
    sigma = 0.7
    return tf.abs(tf.reduce_mean(y_pred) - sigma)

@tf.keras.utils.register_keras_serializable()
class Selector(tf.keras.Model):
    def __init__(self, input_dim=2048, hidden_dim=512, **kwargs):
        super(Selector, self).__init__(**kwargs)
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(hidden_dim, return_sequences=True, dtype="float32")
        )
        self.output_layer = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(1, activation='sigmoid', dtype="float32")
        )

    def call(self, inputs):
        x = self.bilstm(inputs)
        return self.output_layer(x)

    def get_config(self):
        config = super().get_config()
        config.update({"input_dim": 2048, "hidden_dim": 512})
        return config

# ========= 2️⃣ Load Trained Selector Model =========
SELECTOR_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/selector_model.keras"

with tf.keras.utils.custom_object_scope({"Selector": Selector, "sparsity_loss": sparsity_loss}):
    selector = tf.keras.models.load_model(SELECTOR_PATH, compile=False)

print("\n✅ Selector model loaded successfully")

# ========= 3️⃣ Helper Functions =========
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    cps = algo.predict(pen=penalty)
    segments, start = [], 0
    for cp in cps:
        segments.append((start, cp - 1))
        start = cp
    if segments[-1][1] < features.shape[0] - 1:
        segments.append((start, features.shape[0] - 1))
    return segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]

# ========= 4️⃣ TVSum Dataset Paths =========
FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/TVSUM/RandomSplit_Dataset/resnet_npy/extracted_features_25fps_videos"
MAT_FILE_PATH = "/mnt/d/Mass Projects/Video_Summ/ydata-tvsum50-v1_1/ydata-tvsum50-matlab/matlab/ydata-tvsum50.mat"
EXCEL_OUT = "/mnt/d/Mass Projects/Video_Summ/RESULTS/RESULTS/TVSUM/selector_model_tvsum_eval.xlsx"
FPS = 25
PENALTY = 500
SEQ_LEN = 150

# ========= 5️⃣ Load TVSum .mat File =========
with h5py.File(MAT_FILE_PATH, "r") as mat_data:
    tvsum50_group = mat_data["tvsum50"]

    def decode_str(ref):
        return "".join(map(chr, mat_data[ref][()].flatten())).replace("\x00", "")

    video_names = [decode_str(ref) for ref in tvsum50_group["video"][:, 0]]
    user_annos = [mat_data[ref][:] for ref in tvsum50_group["user_anno"][:, 0]]
    nframes_list = [int(mat_data[ref][()][0, 0]) for ref in tvsum50_group["nframes"][:, 0]]

# ========= 6️⃣ Evaluate on All Videos =========
with pd.ExcelWriter(EXCEL_OUT, engine="xlsxwriter") as writer:
    summary_data = []

    for idx, video_name in enumerate(video_names):
        print(f"\n🚀 Evaluating video: {video_name}")

        feature_path = os.path.join(FEATURES_DIR, f"{video_name}.npy")
        if not os.path.exists(feature_path):
            print(f"❌ Missing feature file: {video_name}")
            continue

        raw_feats = np.load(feature_path)
        num_frames = raw_feats.shape[0]

        # Predict importance for sequences
        seqs = []
        for start in range(0, num_frames - SEQ_LEN + 1, SEQ_LEN):
            seq = raw_feats[start:start + SEQ_LEN]
            seqs.append(seq)
        if not seqs:
            continue
        seqs = np.array(seqs, dtype=np.float32)
        preds = selector.predict(seqs, verbose=0)
        preds = preds.reshape(-1)[:num_frames]
        imp_scores = (preds - preds.min()) / (preds.max() - preds.min() + 1e-8)

        # KTS segmentation + knapsack
        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        # Evaluate user annotations
        user_scores = user_annos[idx]
        num_users = user_scores.shape[0]
        all_user_metrics = []

        for u in range(num_users):
            user_score = user_scores[u]
            k = int(np.floor(num_frames * 0.15))
            selected_indices = np.argsort(user_score)[-k:]
            user_segments = get_segments_from_indices(selected_indices.tolist())

            f1, precision, recall = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=video_name[:31], index=False)

        summary_data.append({
            "Video": video_name,
            "KTS Segments": len(model_segments),
            "Selected": len(selected_model_segments),
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max(),
        })

        del seqs
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ All results saved to: {EXCEL_OUT}")


# PLot image of top 10 frames

In [ ]:
import os
import numpy as np
import tensorflow as tf
import scipy.io
import ruptures as rpt
import pandas as pd
import gc
import matplotlib.pyplot as plt
from PIL import Image

# ========= 1. Load Model =========
class BiLSTMAttentionModel(tf.keras.Model):
    def __init__(self, feature_dim=2048, num_heads=8, lstm_units=128, **kwargs):
        super(BiLSTMAttentionModel, self).__init__(**kwargs)
        self.bilstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(lstm_units, return_sequences=True))
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=feature_dim // num_heads)
        self.reconstruction_layer = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(feature_dim, activation="linear"))

    def call(self, inputs):
        x = self.bilstm(inputs)
        x = self.attention(x, x)
        return self.reconstruction_layer(x)

MODEL_PATH = "/mnt/d/Mass Projects/Video_Summ/RESULTS/MODELS/bilstm_multi_2-heads_60k.keras"
with tf.keras.utils.custom_object_scope({"BiLSTMAttentionModel": BiLSTMAttentionModel}):
    model = tf.keras.models.load_model(MODEL_PATH)
print("\n✅ Model loaded successfully")

# ========= 2. Helper Functions =========
def get_segments_from_indices(indices):
    if not indices:
        return []
    indices = sorted(set(indices))
    segments, start = [], indices[0]
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            segments.append((start, indices[i - 1]))
            start = indices[i]
    segments.append((start, indices[-1]))
    return segments

def expand_segment(seg):
    return list(range(seg[0], seg[1] + 1))

def calculate_segment_f1(user_segments, model_segments):
    tp = 0
    matched = []
    for m_seg in model_segments:
        m_range = set(expand_segment(m_seg))
        for u_seg in user_segments:
            u_range = set(expand_segment(u_seg))
            if m_range & u_range:
                tp += 1
                matched.append(m_seg)
                break
    precision = tp / len(model_segments) if model_segments else 0
    recall = tp / len(user_segments) if user_segments else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return f1, precision, recall, matched

def detect_kts_segments(features, penalty=500):
    algo = rpt.KernelCPD(kernel="linear").fit(features)
    change_points = algo.predict(pen=penalty)
    change_points = [int(cp) for cp in change_points]
    scene_segments = []
    start_frame = 0
    for cp in change_points:
        scene_segments.append((start_frame, cp - 1))
        start_frame = cp
    if scene_segments[-1][1] < features.shape[0] - 1:
        scene_segments.append((start_frame, features.shape[0] - 1))
    return scene_segments

def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(int(capacity * 100) + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(int(capacity * 100) + 1):
            weight = int(weights[i - 1] * 100)
            if weight <= w:
                dp[i][w] = max(values[i - 1] + dp[i - 1][w - weight], dp[i - 1][w])
            else:
                dp[i][w] = dp[i - 1][w]
    selected = []
    w = int(capacity * 100)
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= int(weights[i - 1] * 100)
    return selected[::-1]

# ========= 3. Process Videos =========
video_list = [
    "Jumps", "Base Jumping", "Air_Force_One", "Bearpark_climbing", "Bike Polo",
    "Bus_in_Rock_Tunnel", "car_over_camera", "Car_railcrossing", "Cockpit_Landing", "Cooking",
    "Eiffel Tower", "Excavators river crossing", "Fire Domino", "Kids_playing_in_leaves",
    "Notre_Dame", "Paintball", "paluma_jump", "playing_ball", "Playing_on_water_slide",
    "Saving dolphines", "Scuba", "St Maarten Landing", "Statue of Liberty", "Valparaiso_Downhill"
]

FEATURES_DIR = "/mnt/d/Mass Projects/Video_Summ/extracted_features/"
MAT_DIR = "/mnt/d/Mass Projects/Video_Summ/mat/"
FRAME_DIR = "/mnt/d/Mass Projects/Video_Summ/video_frames"
CHUNK_SIZE = 500
PENALTY = 500
FPS = 25
SUMMARY_EXCEL = "/mnt/d/Mass Projects/Video_Summ/RESULTS/RESULTS/SUMME/delete.xlsx"

with pd.ExcelWriter(SUMMARY_EXCEL, engine="xlsxwriter") as writer:
    summary_data = []

    for VIDEO_NAME in video_list:
        print(f"\n==============================\n🚀 Processing video: {VIDEO_NAME}\n==============================")

        feats_path = os.path.join(FEATURES_DIR, f"{VIDEO_NAME}.npy")
        mat_path = os.path.join(MAT_DIR, f"{VIDEO_NAME}.mat")
        if not os.path.exists(feats_path) or not os.path.exists(mat_path):
            print("❌ Skipping, missing file.")
            continue

        raw_feats = np.load(feats_path)
        num_frames = raw_feats.shape[0]
        video_feats = raw_feats[np.newaxis, ...]

        recon = np.zeros_like(video_feats)
        for i in range((num_frames + CHUNK_SIZE - 1) // CHUNK_SIZE):
            s, e = i * CHUNK_SIZE, min((i + 1) * CHUNK_SIZE, num_frames)
            recon[:, s:e, :] = model.predict(video_feats[:, s:e, :], verbose=0)
        print("✅ Features reconstructed")

        errs = np.mean(np.abs(video_feats - recon), axis=2).flatten()
        imp_scores = (errs - errs.min()) / (errs.max() - errs.min())

        model_segments = detect_kts_segments(raw_feats, penalty=PENALTY)
        segment_durations = [(end - start + 1) / FPS for start, end in model_segments]
        segment_scores = [np.mean(imp_scores[start:end + 1]) for start, end in model_segments]

        total_duration = num_frames / FPS
        summary_duration = total_duration * 0.15
        selected_indices = knapsack(segment_scores, segment_durations, summary_duration)
        selected_model_segments = [model_segments[i] for i in selected_indices]

        mat = scipy.io.loadmat(mat_path)
        user_scores = mat['user_score']
        num_users = user_scores.shape[1]
        all_user_metrics = []

        for u in range(num_users):
            usr = user_scores[:, u]
            nonzero_indices = np.where(usr > 0)[0]
            user_segments = get_segments_from_indices(nonzero_indices.tolist())

            f1, precision, recall, _ = calculate_segment_f1(user_segments, selected_model_segments)
            all_user_metrics.append((u + 1, f1, precision, recall))

        df = pd.DataFrame(all_user_metrics, columns=["User", "F1 Score", "Precision", "Recall"])
        df.to_excel(writer, sheet_name=VIDEO_NAME[:31], index=False)

        summary_data.append({
            "Video Name": VIDEO_NAME,
            "KTS Segments": len(model_segments),
            "Knapsack Selected": len(selected_model_segments),
            "Penalty": PENALTY,
            "Min Precision": df["Precision"].min(),
            "Avg Precision": df["Precision"].mean(),
            "Max Precision": df["Precision"].max(),
            "Min Recall": df["Recall"].min(),
            "Avg Recall": df["Recall"].mean(),
            "Max Recall": df["Recall"].max(),
            "Min F1": df["F1 Score"].min(),
            "Avg F1": df["F1 Score"].mean(),
            "Max F1": df["F1 Score"].max(),
        })

        # ============================ #
        # 📸 Plot Top Frames from Model #
        # ============================ #
        TOP_N = 10
        top_idxs = sorted(np.argsort(imp_scores)[-TOP_N:])

        fig, axs = plt.subplots(1, TOP_N, figsize=(20, 3))
        fig.suptitle(f"Top {TOP_N} Frames Selected by Model - {VIDEO_NAME}", y=1.05)

        for i, idx in enumerate(top_idxs):
            frame_path = os.path.join(FRAME_DIR, VIDEO_NAME, f"frame_{idx}.jpg")
            if os.path.exists(frame_path):
                img = Image.open(frame_path)
                axs[i].imshow(img)
                axs[i].set_title(f"Frame {idx}", fontsize=8)
                axs[i].axis("off")
            else:
                axs[i].axis("off")
                axs[i].set_title(f"Missing {idx}", fontsize=8)

        plt.tight_layout()
        plt.show()

        del video_feats, recon
        gc.collect()
        tf.keras.backend.clear_session()

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    print(f"\n✅ All results saved to: {SUMMARY_EXCEL}")


In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# ============================
# 🔹 User Inputs
# ============================
VIDEO_NAME = "Fire Domino"  # change to your video name
FRAME_DIR = "/mnt/d/Mass Projects/Video_Summ/video_frames"

# 🔹 Manually specify frame indices to plot
# (example: top 10 frames you want)
manual_frames = [120, 350, 460, 800, 950, 1120, 1300, 1346, 1364, 1450]

# ============================
# 🔹 Plotting
# ============================
fig, axs = plt.subplots(1, len(manual_frames), figsize=(20, 3))
fig.suptitle(f"Top {len(manual_frames)} Frames Selected by Model - {VIDEO_NAME}", y=1.05)

for i, idx in enumerate(manual_frames):
    frame_path = os.path.join(FRAME_DIR, VIDEO_NAME, f"frame_{idx}.jpg")
    if os.path.exists(frame_path):
        img = Image.open(frame_path)
        axs[i].imshow(img)
        axs[i].set_title(f"Frame {idx}", fontsize=8)
        axs[i].axis("off")
    else:
        axs[i].axis("off")
        axs[i].set_title(f"Missing {idx}", fontsize=8)

plt.tight_layout()
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# ============================
# 🔹 User Inputs
# ============================
VIDEO_NAME = "Saving dolphines"  # change to your video name
FRAME_DIR = "/mnt/d/Mass Projects/Video_Summ/video_frames"

# 🔹 Manually specify frame indices to plot
# (example: top 10 frames you want)
manual_frames = [1544, 1598, 1839, 2970, 3527, 4008, 4326, 5064, 6097, 6518]

# ============================
# 🔹 Plotting
# ============================
fig, axs = plt.subplots(1, len(manual_frames), figsize=(20, 3))
fig.suptitle(f"Top {len(manual_frames)} Frames Selected by Model - {VIDEO_NAME}", y=1.05)

for i, idx in enumerate(manual_frames):
    frame_path = os.path.join(FRAME_DIR, VIDEO_NAME, f"frame_{idx}.jpg")
    if os.path.exists(frame_path):
        img = Image.open(frame_path)
        axs[i].imshow(img)
        axs[i].set_title(f"Frame {idx}", fontsize=8)
        axs[i].axis("off")
    else:
        axs[i].axis("off")
        axs[i].set_title(f"Missing {idx}", fontsize=8)

plt.tight_layout()
plt.show()
